In [3]:
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

%matplotlib inline

In [5]:
path    = '/data/'  # If data in 'data' directory
delian = pd.read_excel(path + 'DeLiAn_edited.xlsx', sheet_name=None)
# Now delian is a dictionary where each key is the sheet name
# and each value is the corresponding DataFrame

In [6]:
for sheet_name, df in delian.items():
    print(f"Sheet Name: {sheet_name}")
    print(df.head())

Sheet Name: List of variables
   Sheet                            Variables (unit)  \
0      1   angstrom_exponent_backscatter_355_532 (1)   
1      2  angstrom_exponent_backscatter_532_1064 (1)   
2      3    angstrom_exponent_extinction_355_532 (1)   
3      4                                 campaign_rv   
4      5                                        date   

                                         Description  
0  Angstrom exponent at 355/532 nm backscatter-re...  
1  Angstrom exponent at 532/1064 nm backscatter-r...  
2  Angstrom exponent at 355/532 nm extinction-rel...  
3     Name of the campaign or of the research vessel  
4  Date of the lidar observations (DD-MM-YYYY or ...  
Sheet Name: 1
       Ash  Saharan dust  Central Asian dust  Middle Eastern dust   Smoke  \
0 -9999.00       -9999.0           -0.061045              -9999.0 -9999.0   
1 -9999.00       -9999.0           -0.418634                  0.6 -9999.0   
2     0.22       -9999.0           -0.362652              

In [7]:
delian.keys()

dict_keys(['List of variables', '1', '6', '2', '7', '3', '8', '13', '9', '14', '10', '18', '11', '19', '12', '17', '4', '5', '15', '16', '20', '21'])

In [8]:
variable_names = delian['List of variables']
# Iterate through each sheet and rename them
for sheet_name in delian.keys():
    if sheet_name != 'List of variables':
        # Filter the rows based on the condition and selecting the 'Variables (unit)' column
        new_sheet_name = variable_names.loc[variable_names['Sheet'] == int(sheet_name), 'Variables (unit)'].values[0]
        delian[new_sheet_name] = delian.pop(sheet_name)

In [9]:
delian.keys()

dict_keys(['List of variables', 'angstrom_exponent_backscatter_355_532 (1)', 'error_angstrom_exponent_backscatter_355_532 (1)', 'angstrom_exponent_backscatter_532_1064 (1)', 'error_angstrom_exponent_backscatter_532_1064 (1)', 'angstrom_exponent_extinction_355_532 (1)', 'error_angstrom_exponent_extinction_355_532 (1)', 'lidar_ratio_355 (sr)', 'error_lidar_ratio_355 (sr)', 'lidar_ratio_532 (sr)', 'error_lidar_ratio_532 (sr)', 'particle_linear_depolarization_ratio_355 (%)', 'error_particle_linear_depolarization_ratio_355 (%)', 'particle_linear_depolarization_ratio_532 (%)', 'error_particle_linear_depolarization_ratio_532 (%)', 'number_samples (1)', 'campaign_rv', 'date', 'location', 'measurement_type', 'reference', 'system'])

In [10]:
samples = delian['number_samples (1)']
samples

,Ash,Saharan dust,Central Asian dust,Middle Eastern dust,Smoke,Stratospheric smoke,Dust and smoke,Pollution,Dust and pollution,Dried marine,Clean marine,Dust and marine,Central European background
0,4,30,23,4,71,8,25,42,15,7,52,21,14


In [11]:
#Define a new df containing the label (type of aerosol) of all observations
aerosol = []
for col in samples.columns:
  for i in range(int(samples[col])):
    aerosol.append(col)
aerosol = pd.DataFrame({"aerosol": aerosol})
aerosol['aerosol'] = aerosol['aerosol'].astype('category') #convert type from Object to Categorical
aerosol

,aerosol
0,Ash
1,Ash
2,Ash
3,Ash
4,Saharan dust
...,...
311,Central European background
312,Central European background
313,Central European background
314,Central European background


In [12]:
# Sheets to be interessed
quantitatives = [
    'angstrom_exponent_backscatter_355_532 (1)',
    'error_angstrom_exponent_backscatter_355_532 (1)',
    'angstrom_exponent_backscatter_532_1064 (1)',
    'error_angstrom_exponent_backscatter_532_1064 (1)',
    'angstrom_exponent_extinction_355_532 (1)',
    'error_angstrom_exponent_extinction_355_532 (1)',
    'lidar_ratio_355 (sr)',
    'error_lidar_ratio_355 (sr)',
    'lidar_ratio_532 (sr)',
    'error_lidar_ratio_532 (sr)',
    'particle_linear_depolarization_ratio_355 (%)',
    'error_particle_linear_depolarization_ratio_355 (%)',
    'particle_linear_depolarization_ratio_532 (%)',
    'error_particle_linear_depolarization_ratio_532 (%)'
]
delian = {key: delian[key] for key in quantitatives}

In [13]:
# Convert each sheet to a column as variable
delian_df = pd.DataFrame()

# Iterate through each sheet and extract information
for sheet_name, df in delian.items():
  new_values = []
  for col in df.columns:
    for value in df[col]:
      # Check if the value is not NaN
      if pd.notna(value):
        new_values.append(value)
  new_df = pd.DataFrame({sheet_name: new_values})
  delian_df = pd.concat([delian_df, new_df], axis=1)

delian_df = pd.concat([delian_df, aerosol], axis=1)
delian_df

,angstrom_exponent_backscatter_355_532 (1),error_angstrom_exponent_backscatter_355_532 (1),angstrom_exponent_backscatter_532_1064 (1),error_angstrom_exponent_backscatter_532_1064 (1),angstrom_exponent_extinction_355_532 (1),error_angstrom_exponent_extinction_355_532 (1),lidar_ratio_355 (sr),error_lidar_ratio_355 (sr),lidar_ratio_532 (sr),error_lidar_ratio_532 (sr),particle_linear_depolarization_ratio_355 (%),error_particle_linear_depolarization_ratio_355 (%),particle_linear_depolarization_ratio_532 (%),error_particle_linear_depolarization_ratio_532 (%),aerosol
0,-9999.00,-9999.0,-9999.00,-9999.00,-9999.00,-9999.00,55.00,5.00,-9999.0,-9999.0,36.50,1.0,-9999.0,-9999.0,Ash
1,-9999.00,-9999.0,-9999.00,-9999.00,-9999.00,-9999.00,55.00,5.00,-9999.0,-9999.0,37.50,2.0,-9999.0,-9999.0,Ash
2,0.22,0.4,1.05,0.43,0.68,0.63,39.00,10.00,32.0,4.0,-9999.00,-9999.0,-9999.0,-9999.0,Ash
3,1.00,-9999.0,1.00,-9999.00,1.00,-9999.00,55.00,10.00,64.0,11.0,34.00,4.0,-9999.0,-9999.0,Ash
4,-9999.00,-9999.0,-9999.00,-9999.00,-9999.00,-9999.00,63.00,9.00,-9999.0,-9999.0,24.00,2.0,31.0,1.0,Saharan dust
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,-9999.00,-9999.0,-9999.00,-9999.00,-9999.00,-9999.00,48.07,2.24,-9999.0,-9999.0,2.91,2.0,-9999.0,-9999.0,Central European background
312,-9999.00,-9999.0,-9999.00,-9999.00,-9999.00,-9999.00,58.02,7.59,-9999.0,-9999.0,3.76,2.0,-9999.0,-9999.0,Central European background
313,-9999.00,-9999.0,-9999.00,-9999.00,-9999.00,-9999.00,58.57,5.69,-9999.0,-9999.0,3.75,2.0,-9999.0,-9999.0,Central European background
314,-9999.00,-9999.0,-9999.00,-9999.00,-9999.00,-9999.00,60.56,3.41,-9999.0,-9999.0,3.91,2.0,-9999.0,-9999.0,Central European background


In [19]:
test = delian_df.copy()

# Hàm tính trung bình không tính đến -9999
def mean_without_9999(column):
    mean = column[column != -9999.].mean()
    return mean

# Hàm thay thế giá trị -9999 bằng giá trị trung bình của các giá trị còn lại của biến
def replace_9999(column):
    mean = mean_without_9999(column)
    column[column == -9999.] = mean
    return column

# Áp dụng hàm replace_9999 cho mỗi biến của mỗi loại aerosol trong DataFrame
df = test.groupby('aerosol').transform(replace_9999)

df

,angstrom_exponent_backscatter_355_532 (1),error_angstrom_exponent_backscatter_355_532 (1),angstrom_exponent_backscatter_532_1064 (1),error_angstrom_exponent_backscatter_532_1064 (1),angstrom_exponent_extinction_355_532 (1),error_angstrom_exponent_extinction_355_532 (1),lidar_ratio_355 (sr),error_lidar_ratio_355 (sr),lidar_ratio_532 (sr),error_lidar_ratio_532 (sr),particle_linear_depolarization_ratio_355 (%),error_particle_linear_depolarization_ratio_355 (%),particle_linear_depolarization_ratio_532 (%),error_particle_linear_depolarization_ratio_532 (%)
0,0.610000,0.400000,1.025000,0.430000,0.840000,0.630000,55.00,5.00,48.00000,7.500000,36.50,1.000000,NaN,NaN
1,0.610000,0.400000,1.025000,0.430000,0.840000,0.630000,55.00,5.00,48.00000,7.500000,37.50,2.000000,NaN,NaN
2,0.220000,0.400000,1.050000,0.430000,0.680000,0.630000,39.00,10.00,32.00000,4.000000,36.00,2.333333,NaN,NaN
3,1.000000,0.400000,1.000000,0.430000,1.000000,0.630000,55.00,10.00,64.00000,11.000000,34.00,4.000000,NaN,NaN
4,0.028737,0.086699,0.459035,0.117678,0.100489,0.166344,63.00,9.00,53.06897,7.863113,24.00,2.000000,31.000,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,1.444033,0.201567,1.234233,0.189733,1.492750,0.177300,48.07,2.24,56.19020,8.294167,2.91,2.000000,3.165,0.0935
312,1.444033,0.201567,1.234233,0.189733,1.492750,0.177300,58.02,7.59,56.19020,8.294167,3.76,2.000000,3.165,0.0935
313,1.444033,0.201567,1.234233,0.189733,1.492750,0.177300,58.57,5.69,56.19020,8.294167,3.75,2.000000,3.165,0.0935
314,1.444033,0.201567,1.234233,0.189733,1.492750,0.177300,60.56,3.41,56.19020,8.294167,3.91,2.000000,3.165,0.0935


In [24]:
df.duplicated().sum()

0

In [28]:
df.isna().sum()

angstrom_exponent_backscatter_355_532 (1)             0
error_angstrom_exponent_backscatter_355_532 (1)       0
angstrom_exponent_backscatter_532_1064 (1)            0
error_angstrom_exponent_backscatter_532_1064 (1)      0
angstrom_exponent_extinction_355_532 (1)              0
error_angstrom_exponent_extinction_355_532 (1)        0
lidar_ratio_355 (sr)                                  0
error_lidar_ratio_355 (sr)                            0
lidar_ratio_532 (sr)                                  0
error_lidar_ratio_532 (sr)                            0
particle_linear_depolarization_ratio_355 (%)          0
error_particle_linear_depolarization_ratio_355 (%)    0
particle_linear_depolarization_ratio_532 (%)          4
error_particle_linear_depolarization_ratio_532 (%)    4
dtype: int64